# local_planner.py 算法流程介绍

简洁按模块介绍 Layer 2 的轨迹生成算法：

## 整体流程



In [ ]:
几何路径 → 路径平滑 → 曲率计算 → 速度规划 → 轨迹构建 → 停止段
  (input)   (B样条)    (解析求导)  (梯形曲线)  (时间戳)  (output)



---

## 1. 路径平滑 (`_smooth_path_spline`)

**目标**：将 A* 的折线路径转换为光滑曲线

**方法**：
- 计算累积弧长，归一化到 [0,1]
- 用 `UnivariateSpline(degree=3, s=smoothing)` 拟合 x(s), y(s)
  - `s=0`：精确插值（过所有点）
  - `s>0`：B样条逼近（更平滑）
- **解析求导**计算航向角 θ 和曲率 κ：
  ```
  θ = arctan2(dy/ds, dx/ds)
  κ = (dy/ds·d²x/ds² - dx/ds·d²y/ds²) / (dx²+dy²)^(3/2)
  ```

**输出**：密集路径点 + 解析航向 + 解析曲率（缓存备用）

---

## 2. 曲率计算 (`_compute_curvatures`)

**备用方法**（当样条失败时）

**方法**：三点法
- 相邻三点的角度差 / 弧长 = 曲率
- 滑动窗口平滑（5点均值）

---

## 3. 速度规划 (`_plan_speeds_optimized`) ⭐ 核心

**目标**：生成满足**加速度约束**的梯形速度曲线

**四步骤**：

### Step 1: 曲率速度上限


In [ ]:
v_curvature[i] = v_max / √(1 + k·|κ|)

- 直道：κ≈0 → v≈v_max
- 弯道：κ↑ → v↓（安全转弯）

### Step 2: 终点速度为 0


In [ ]:
v[-1] = v[-2] = 0  （停下来）



### Step 3: 后向传播（减速约束）


In [ ]:
v[i]² ≤ v[i+1]² + 2·a_max·ds

确保能从当前速度减速到下一点

### Step 4: 前向传播（加速约束）


In [ ]:
v[i+1]² ≤ v[i]² + 2·a_max·ds

确保能从当前速度加速到下一点

**结果**：梯形速度曲线（启动→巡航→减速）

---

## 4. 轨迹构建 (`_build_trajectory_improved`)

**关键**：每个点包含 `{x, y, θ, v, ω, t}`

**逐点处理**：
- **航向角 θ**：优先用解析值（更平滑），否则用差分
- **角速度 ω**：`ω = Δθ / Δt`（限制在 [-w_max, +w_max]）
- **时间戳 t**：
  ```
  t[i+1] = t[i] + ds[i] / v[i]  （基于实际距离和速度）
  ```

---

## 5. 停止段 (`_add_stop_segment`)

**目标**：轨迹末尾添加静止状态，让控制器平稳停止

**操作**：复制最后一个点（v=0, ω=0）若干次

---

## 关键参数

| 参数 | 作用 |
|------|------|
| `v_max / v_min` | 速度上下限 |
| `w_max` | 最大角速度（用于裁剪 ω） |
| `a_max` | 最大线加速度（梯形曲线关键） |
| `alpha_max` | 最大角加速度（目前未用） |
| `use_bspline=True` | B样条逼近（vs 精确插值） |
| `spacing=0.05` | 采样间距 (m) |
| `curvature_gain=3.0` | 曲率敏感度 |

---

## 优点总结

✅ **解析求导**：航向/曲率准确、光滑  
✅ **梯形加减速**：满足物理约束  
✅ **弯道减速**：基于曲率自适应  
✅ **时间参数化**：直接给 MPC/MPPI 用  

---

## 与其他层的关系



In [ ]:
Layer1 (A*) → 折线路径
    ↓
Layer2 (local_planner) ← 你在这儿
    ↓ [产出参考轨迹]
Layer3 (MPC/MPPI) → 跟踪控制